### 
- Multi level sales reporting 
- Total sales per city
- total sales per products
- total sales per each quarter
- total sales per city, product, quarter
- grand total 



In [0]:
%sql

create table postg.products(prod_id int, prod_name string);
insert into postg.products values(1,"apple");
insert into postg.products values(2,"samsung");
insert into postg.products values(3,"redmi");


In [0]:
%sql

create table postg.sales(sale_id int ,prod_id int, city string, sale_date date, amount double);
insert into postg.sales values (1 ,1, "Hyd", "2025-01-05", 10000);
insert into postg.sales values (2 ,2, "Pune", "2025-02-25", 30000);
insert into postg.sales values (3 ,3, "Hyd", "2025-02-15", 40000);
insert into postg.sales values (4 ,1, "Pune", "2025-01-11", 50000);
insert into postg.sales values (5 ,1, "Hyd", "2025-03-05", 60000);
insert into postg.sales values (6 ,2, "Mum", "2025-04-05", 70000);
insert into postg.sales values (7 ,3, "Hyd", "2025-05-05", 80000);
insert into postg.sales values (8 ,1, "Pune", "2025-06-05", 90000);
insert into postg.sales values (9 ,1, "Hyd", "2025-07-05", 15000);
insert into postg.sales values (10 ,1, "Pune", "2025-08-05", 16000);
insert into postg.sales values (11 ,2, "Hyd", "2025-09-05", 17000);
insert into postg.sales values (12 ,2, "Mum", "2025-10-05", 18000);
insert into postg.sales values (13 ,2, "Hyd", "2025-11-05", 19000);
insert into postg.sales values (14 ,3, "Guntur", "2025-01-15", 200000);
insert into postg.sales values (15 ,1, "Hyd", "2025-01-25", 21000);
insert into postg.sales values (16 ,3, "Guntur", "2025-03-25", 22000);
insert into postg.sales values (17 ,3, "Hyd", "2025-01-05", 23000);
insert into postg.sales values (18 ,2, "Guntur", "2025-04-15", 24000);
insert into postg.sales values (19 ,2, "Hyd", "2025-08-09", 25000);
insert into postg.sales values (20 ,1, "Guntur", "2025-06-27", 350000);

In [0]:
%sql select * from postg.sales;

In [0]:
%sql 
use postg;
select city, sum(amount) as total_sales from products join sales on sales.prod_id=products.prod_id group by city

In [0]:
prddf = spark.read.table('postg.products')
salesdf = spark.read.table("postg.sales")
#display(prddf)
#display(salesdf)

In [0]:
# city sales count

from pyspark.sql.functions import * 
from pyspark.sql.types import *

prddf.join(salesdf, prddf.prod_id == salesdf.prod_id,'inner').groupBy("city").agg(sum(salesdf.amount).alias("total_sales")).show()



In [0]:
%sql
use postg;

select *, extract(quarter from sale_date) from sales;

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import quarter

salesdf.withColumn('quarter', quarter(salesdf['sale_date'])).show()

In [0]:
%sql
select city,prod_name, sum(amount) as total_sales,count(1) as num_sales from products join sales on sales.prod_id=products.prod_id group by city,prod_name

In [0]:
from pyspark.sql.functions import col
salesdf.join(prddf,salesdf.prod_id == prddf.prod_id,"inner").\
    groupBy(col('city'),col('prod_name')).\
        agg(count(col('*')).alias('num_sales'),sum(col('amount')).alias('total_sales')).show()


In [0]:
%sql
select city,prod_name, sum(amount) as total_sales
from products join sales on sales.prod_id=products.prod_id 
group by grouping sets(city,prod_name,())
--group by grouping sets ((city),(prod_name),(city,prod_name),())

In [0]:
from pyspark.sql.functions import *

df =salesdf.join(prddf, prddf.prod_id == salesdf.prod_id,"inner").\
groupingSets([("city",),("prod_name",),()],"city","prod_name").\
    agg(sum(salesdf.amount))
display(df)    

 #[("city", "car_model"), ("city",), ()],"city", "car_model"

In [0]:
%sql
select city,prod_name, sum(amount) as total_sales,
grouping(city) as group_city,
grouping(prod_name) as group_prod_name
from products join sales on sales.prod_id=products.prod_id 
group by grouping sets(city,prod_name,())
--group by grouping sets ((city),(prod_name),(city,prod_name),())

In [0]:
salesdf.join(prddf, prddf.prod_id == salesdf.prod_id,"inner").\
    groupingSets([("city",),("prod_name",),()],"city","prod_name").\
        agg(sum(salesdf.amount).alias("total_sales")).show()


In [0]:
%sql 
use postg;

select city,prod_name, extract(quarter from sale_date) as qtr ,sum(amount) as total_sales,
grouping(city)  as group_city,
grouping(prod_name) as group_product
from sales 
join products 
on sales.prod_id = products.prod_id
group by grouping sets (city, prod_name,extract(quarter from sale_date) , ())